In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import scipy as sp

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context("poster")
sns.set_style("ticks")
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import silhouette_score
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import scipy.cluster.hierarchy as shc

from kmodes.kmodes import KModes

In [ ]:
data_path = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/PeskindTBI/final_data/first_visits_short.csv'

In [ ]:
#read in csv containing data from all surveys and all visitseqs
data = pd.read_csv(data_path, index_col=0)
data = pd.DataFrame(data = data)

print('Original data shape:\n', data.shape, '\n')
print(data.info())
data.head(1)

In [ ]:
#determined outliers for auditc and QBlstExp (outlier = >3 SD from mean) and remove
data = data[data["TBIID"] != 'C010']
data = data[data["TBIID"] != 'T080']

In [ ]:
#create variable lists depending on data type for graphing purposes

data_cont = ['ScreenAge', 'Education', 'servconn', 'cestotal',
       'MnthSncBlst', 'QKOIorA', 'QKOExpMil', 'QKOAllMil', 'QKOLife',
       'QBlstExp', 'QBEACRM', 'Q5plus2', 'QEDist_sum', 'QEDist_mean',
       'QEDist_min', 'DOPA', 'DA', 'DOPAC', 'NE', 'DHPG', 'da_dopa_ratio', 'dopac_da_ratio',
       'ne_dopa_ratio', 'dhpg_ne_ratio', 'HEIGHT', 'WEIGHT', 'BMI',
       'BPSYS', 'BPDIAS', 'HRATE', 'BGlucose', 'BNa', 'BUN', 'BCreat', 'UNa',
       'BK', 'TotalChol', 'LDL', 'HDL', 'Trig', 'CSFPROTEIN_x',
       'CSFGLUCOSE', 'CSFRBCS',  
       'CAPSTotal', 'PSQIc1', 'PSQIc2', 'PSQIc3', 'PSQIc4', 'PSQIc5',
       'PSQIc6', 'PSQIc7', 'PSQItot', 'PCLTot', 'PCL_reexp', 'PCL_avoid',
       'PCL_numb', 'PCL_hyper', 'PHQTot', 'PHQ_psych', 'PHQ_somatic',
       'NSITot', 'TBITot', 'NSI_vestibular', 'NSI_somatosensory',
       'NSI_cognitive', 'NSI_affective', 'NSI_ERP_affective',
       'NIS_ERP_vestsom', 'DvpHA', 'DvpHAAct', 'DvpHASlp',
       'DvpHAMd', 'DvpHAStr', 'DvpBP', 'DvpBPAct', 'DvpBPslp', 'DvpBPMd',
       'DvpBPStr', 'BISAtt', 'BISMtr', 'BISNonpl', 'BISTot',
             'PreSleep', 'PreCaff', 'PreETOH', 'PreNic', 'PreTHC']

data_cat = ['Status_x', 'VisitSeq', 'GType', 'Race', 'Hispanic', 'Handedness', 'Marital', 'APOEGen', 
           'HYPERTEN', 'HYPERCHO', 'DIABETES', 'B12DEF', 'THYROID',
           'MHxPain', 'MHxHA', 'MHxHtn', 'MHxCard', 'MHxGI', 'MHxNeuro', 'MHxLung', 'MHxApnea', 
            'SCPTSD', 'SCMDD', 'SCPD', 'SCGAD', 'SCNone', 'capsCrtA']

data_audit = ['auditc', 'AUDIT1', 'AUDIT2', 'AUDIT3']

## AUDIT-C clustering with C and T from first visit sequence

In [ ]:
#perform clustering on individual auditc questions (3 total questions) from visit sequence 1
data_first = data[data['VisitSeq'] == 1]
data_allgroups_auditqs = data_first[['VisitSeq', 'Group', 'TBIID', 'AUDIT1', 'AUDIT2', 'AUDIT3']].set_index(['VisitSeq', 'Group', 'TBIID'])
print(data_allgroups_auditqs.shape)
data_allgroups_auditqs.dropna(axis=0, inplace=True)
print(data_allgroups_auditqs.shape)
data_allgroups_auditqs.head()

In [ ]:
# center and scale the data
scaler = StandardScaler()

data_allgroups_auditqs_scaled = scaler.fit_transform(data_allgroups_auditqs)

In [ ]:
#viz dendrogram to find if three clusters makes sense
plt.figure(figsize=(10, 7))  
plt.title("AUDIT-C 3 Question Dendrogram")  
plt.ylabel("Distance (dissimilarity)")
plt.xlabel("Participants")
dend = shc.dendrogram(shc.linkage(data_allgroups_auditqs, method='ward'), 
                      distance_sort='ascending',
                      show_leaf_counts=True, leaf_font_size=8)

In [ ]:
#pick cluster number based on silhouette coefficient
k_range = range(2,7)
scores = []
for k in k_range:
    km_ss = KMeans(n_clusters=k, random_state=10)
    km_ss.fit(data_allgroups_auditqs)
    scores.append(silhouette_score(data_allgroups_auditqs, km_ss.labels_))

# plot the results
plt.plot(k_range, scores)
plt.title('AUDIT-C Questions K means clustering')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')

In [ ]:
#pick cluster number based on minimizing sse
k_range = range(2,7)
scores = []
for k in k_range:
    km_ss = KMeans(n_clusters=k, random_state=10)
    km_ss.fit(data_allgroups_auditqs)
    scores.append(km_ss.inertia_)
    
# plot the results
plt.plot(k_range, scores)
plt.title('AUDIT-C Questions K means clustering')
plt.xlabel('Number of clusters')
plt.ylabel('SSE')

In [ ]:
#viz clustering with heat map of AUDIT-C answers
sns.clustermap(data_allgroups_auditqs, metric="euclidean", standard_scale=1, method="ward", cmap="Blues")

In [ ]:
#choose k=3 clusters and fit data
km_3 = KMeans(n_clusters=3,random_state=1)
km_3.fit(data_allgroups_auditqs)
#new df for cluster info
data_allgroups_auditqs_clusters = data_allgroups_auditqs
data_allgroups_auditqs_clusters = data_allgroups_auditqs_clusters.reset_index()
data_allgroups_auditqs_clusters['kmeans_cluster'] = ["cluster_" + str(label) for label in km_3.labels_ ]
data_allgroups_auditqs_clusters.head(1)

In [ ]:
#merge with orig df so each TBIID entry has its corr cluster assignment added
data_clusters = pd.merge(data, data_allgroups_auditqs_clusters[['TBIID', 'kmeans_cluster']], on=['TBIID'], how='outer')
print(data_clusters.shape)
print(data_clusters['VisitSeq'].value_counts())
data_clusters.head(1)

In [ ]:
#viz cluster grouping and auditc answers - seems to be low(1), high freq(2), and binge clusters(0)
auditc_cols = ['auditc', 'AUDIT1', 'AUDIT2', 'AUDIT3']
for variable in auditc_cols:
    try:
        plt.figure(figsize=(7,7))
        g = sns.barplot(x='kmeans_cluster', y=variable, data=data_clusters[data_clusters['VisitSeq'] == 1], ci=68, hue='Group', palette="rocket", order=["cluster_0", "cluster_1", "cluster_2"])
        plt.show()
    except:
        pass

In [ ]:
#rename clusters for easier viz understanding
data_clusters.replace({'kmeans_cluster': {'cluster_0':'binge', 'cluster_1':'low', 'cluster_2':'freq'}}, inplace=True)
#look at counts in each cluster for each group (is there a difference in cluster patterns between groups?)
data_clusters[data_clusters['VisitSeq'] == 1].groupby(['Group'])['kmeans_cluster'].value_counts().reset_index(name='count').sort_values(['Group', 'kmeans_cluster'])

In [ ]:
tsne = TSNE(n_components=2, perplexity=50, random_state=1234)
tsne_features = tsne.fit_transform(data_allgroups_auditqs)

print(tsne_features.shape)
tsne_df = pd.DataFrame(data = tsne_features, columns = ['tsne_0', 'tsne_1'], index = data_allgroups_auditqs_clusters.index)
data_allgroups_auditqs_clusters_TSNE = pd.concat([data_allgroups_auditqs_clusters, tsne_df], axis = 1)
data_allgroups_auditqs_clusters_TSNE.replace({'kmeans_cluster': {'cluster_0':'binge', 'cluster_1':'low', 'cluster_2':'freq'}}, inplace=True)
data_allgroups_auditqs_clusters_TSNE.head()

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x='tsne_0', y='tsne_1', data=data_allgroups_auditqs_clusters_TSNE, hue='kmeans_cluster')

### TBI only add'n analysis and viz

In [ ]:
#select just TBI participants and exclude visit '88' for add'l analysis
data_clusters_TBI = data_clusters[(data_clusters['Group'] == 'T') & (data_clusters['VisitSeq'] != 88)]

In [ ]:
#viz for first visit seq

data_viz = data_clusters_TBI

for param in data_cat:
    print(param)

    data_int = (data_viz[data_viz[param] != 9].groupby('kmeans_cluster')[param].value_counts() /
                data_viz[data_viz[param] != 9].groupby('kmeans_cluster')[param].count()).reset_index(name='perc')
        
    try:
        g = sns.catplot(x=param, y='perc', kind='bar', data=data_int, hue='kmeans_cluster', ci=68, height=5, aspect=4)
        plt.show()
        
        print('\n')
        
    except:
        pass
    
for param in data_audit:
    print(param)

    data_int = (data_viz.groupby('kmeans_cluster')[param].value_counts() /
                        data_viz.groupby('kmeans_cluster')[param].count()).reset_index(name='perc')
        
    try:
        g = sns.catplot(x=param, y='perc', kind='bar', data=data_int, hue='kmeans_cluster', ci=68, height=5, aspect=4)
        plt.show()
        
        print('\n')
        
    except:
        pass
    
for param in data_cont:
    print(param)
    
    try:
        g = sns.catplot(x='kmeans_cluster', y=param, kind='bar', data=data_viz, ci=68, height=5, aspect=4)
        plt.show()
        
        print('\n')
        
    except:
        pass

In [ ]:
#does drinking cluster affect follow-up visits
print(data_clusters_TBI.groupby(['VisitSeq', 'kmeans_cluster'])['kmeans_cluster'].count())
sns.countplot(x='VisitSeq', data=data_clusters_TBI, hue='kmeans_cluster', palette="rocket")

In [ ]:
#exclude 3rd visit seq
data_clusters_TBI_v12 = data_clusters_TBI[data_clusters_TBI['VisitSeq'] != 3]
#get TBIIDs of participants with a second visit and use to filter
TBIID_v2 = data_clusters_TBI_v12[data_clusters_TBI_v12['VisitSeq'] == 2]['TBIID'].values

#create new column for if came to second visitseq - want to see if params at visitseq 1 map on to whether they came back again
data_clusters_TBI_v12['visit2'] = [1 if TBIID in TBIID_v2 else 0 for TBIID in data_clusters_TBI_v12['TBIID']]

#create new df with only participants who came to both visits 1 and 2
data_clusters_TBI_v12_only = data_clusters_TBI_v12[data_clusters_TBI_v12['TBIID'].isin(TBIID_v2)]
print(data_clusters_TBI_v12_only.shape)
data_clusters_TBI_v12_only.head()

In [ ]:
#viz for first visit seq

data_viz = data_clusters_TBI_v12_only

for param in data_cont:
    print(param)
    
    try:
        g = sns.catplot(x='VisitSeq', y=param, kind='bar', hue='kmeans_cluster', data=data_viz, ci=68, height=5, aspect=4)
        plt.show()
        
        print('\n')
        
    except:
        pass

In [ ]:
#select only visit 1
data_clusters_visit1 = data_clusters[data_clusters['VisitSeq'] == 1]
data_clusters_visit1.to_csv('data_auditc_clustering_visit1.csv')

In [ ]:
#get AUDIT counts and percents for chi squared analysis and viz
AUDIT_tot_counts = data_clusters_visit1.groupby('Group')['auditc'].value_counts()
AUDIT_1_counts = data_clusters_visit1.groupby('Group')['AUDIT1'].value_counts()
AUDIT_2_counts = data_clusters_visit1.groupby('Group')['AUDIT2'].value_counts()
AUDIT_3_counts = data_clusters_visit1.groupby('Group')['AUDIT3'].value_counts()
AUDIT_tot_counts.to_csv('AUDIT_tot_counts.csv')
AUDIT_1_counts.to_csv('AUDIT_1_counts.csv')
AUDIT_2_counts.to_csv('AUDIT_2_counts.csv')
AUDIT_3_counts.to_csv('AUDIT_3_counts.csv')

AUDIT_tot_perc = data_clusters_visit1.groupby('Group')['auditc'].value_counts() / data_clusters_visit1.groupby('Group')['auditc'].count()
AUDIT_1_perc = data_clusters_visit1.groupby('Group')['AUDIT1'].value_counts() / data_clusters_visit1.groupby('Group')['AUDIT1'].count()
AUDIT_2_perc = data_clusters_visit1.groupby('Group')['AUDIT2'].value_counts() / data_clusters_visit1.groupby('Group')['AUDIT2'].count()
AUDIT_3_perc = data_clusters_visit1.groupby('Group')['AUDIT3'].value_counts() / data_clusters_visit1.groupby('Group')['AUDIT3'].count()
AUDIT_tot_perc.to_csv('AUDIT_tot_perc.csv')
AUDIT_1_perc.to_csv('AUDIT_1_perc.csv')
AUDIT_2_perc.to_csv('AUDIT_2_perc.csv')
AUDIT_3_perc.to_csv('AUDIT_3_perc.csv')

In [ ]:
#get value counts for AUDIT-C for each cluster (not split by group)
AUDIT_tot_counts_cluster = data_clusters_visit1.groupby(['kmeans_cluster'])['auditc'].value_counts()
AUDIT_1_counts_cluster = data_clusters_visit1.groupby(['kmeans_cluster'])['AUDIT1'].value_counts()
AUDIT_2_counts_cluster = data_clusters_visit1.groupby(['kmeans_cluster'])['AUDIT2'].value_counts()
AUDIT_3_counts_cluster = data_clusters_visit1.groupby(['kmeans_cluster'])['AUDIT3'].value_counts()
AUDIT_tot_counts_cluster.to_csv('AUDIT_tot_counts_cluster.csv')
AUDIT_1_counts_cluster.to_csv('AUDIT_1_counts_cluster.csv')
AUDIT_2_counts_cluster.to_csv('AUDIT_2_counts_cluster.csv')
AUDIT_3_counts_cluster.to_csv('AUDIT_3_counts_cluster.csv')

AUDIT_tot_perc_cluster = data_clusters_visit1.groupby(['kmeans_cluster'])['auditc'].value_counts() / data_clusters_visit1.groupby(['kmeans_cluster'])['auditc'].count()
AUDIT_1_perc_cluster = data_clusters_visit1.groupby(['kmeans_cluster'])['AUDIT1'].value_counts() / data_clusters_visit1.groupby(['kmeans_cluster'])['AUDIT1'].count()
AUDIT_2_perc_cluster = data_clusters_visit1.groupby(['kmeans_cluster'])['AUDIT2'].value_counts() / data_clusters_visit1.groupby(['kmeans_cluster'])['AUDIT2'].count()
AUDIT_3_perc_cluster = data_clusters_visit1.groupby(['kmeans_cluster'])['AUDIT3'].value_counts()/ data_clusters_visit1.groupby(['kmeans_cluster'])['AUDIT3'].count()
AUDIT_tot_perc_cluster.to_csv('AUDIT_tot_perc_cluster.csv')
AUDIT_1_perc_cluster.to_csv('AUDIT_1_perc_cluster.csv')
AUDIT_2_perc_cluster.to_csv('AUDIT_2_perc_cluster.csv')
AUDIT_3_perc_cluster.to_csv('AUDIT_3_perc_cluster.csv')

#get value counts by group for each cluster
AUDIT_cluster_counts = data_clusters_visit1.groupby('Group')['kmeans_cluster'].value_counts() 
AUDIT_cluster_perc = data_clusters_visit1.groupby('Group')['kmeans_cluster'].value_counts() / data_clusters_visit1.groupby('Group')['kmeans_cluster'].count()
AUDIT_cluster_counts.to_csv('AUDIT_cluster_counts.csv')
AUDIT_cluster_perc.to_csv('AUDIT_cluster_perc.csv')